In [1]:
from bs4 import BeautifulSoup
import requests, re

col_ga_id = []
col_company = []
col_item_name = []
col_sales_price = []
col_origin_price = []
col_discounts = []

col_page =[]

p_company = re.compile(r'\[(?P<company>.*)\](?P<item_name>.*)')
p_price = re.compile(r'\d.*\d')

col_category  = [    
    #밑반찬
    '46010000',
    #메인반찬
    '46030000',
    #국찌개탕
    '46060000',
    #아이반찬
    '46070000',
    #육류
    '46100000',
    #김치장아찌
    '46020000',
    #세계음식
    '46040000',
    #밥죽면
    '46050000',
    #샐러드
    '46080000',
    #간식
    '46120000',
    #정기식단
    '46110000',        
]

col_url = [
    'https://www.baeminchan.com/sidedish/list.php',
    'https://www.baeminchan.com/maindish/list.php',
    'https://www.baeminchan.com/soups/list.php',
    'https://www.baeminchan.com/forkids/list.php',
    'https://www.baeminchan.com/meat/list.php',
    'https://www.baeminchan.com/pickles/list.php',
    'https://www.baeminchan.com/foreign/list.php',
    'https://www.baeminchan.com/rice-noodle/list.php',
    'https://www.baeminchan.com/salad/list.php',
    'https://www.baeminchan.com/snack/list.php',
    'https://www.baeminchan.com/mealplan/list.php',    
]  

for col in col_url:
    response = requests.get(col)
    soup = BeautifulSoup(response.text, 'lxml')    
    try:
        soup.find('div', 'pagination').findAll('a')
    except:
        col_page.append(0)
    else:
        col_page.append(len(soup.find('div', 'pagination').findAll('a'))-1)     

cnt = 0

for cnt in range(0, 11):
    col_url_page = []
    
    if col_page[cnt] == 0:
        col_url_page.append(col_url[cnt])
    else:            
        for num in range(1, col_page[cnt]+1):                    
            col_url_page.append(col_url[cnt]+'?'+'cno='+str(col_category[cnt])+'&'+'page='+str(num))

    for url in col_url_page:    
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        soup_products = soup.find('ul', id='products')
        col_products = []
        col_products = soup_products.findAll('li')
        col_imgthumb = []
        col_imgthumb = soup_products.find_all("div", "imgthumb")

        for col in col_products:
            col_ga_id.append(col.select('a')[0].get('ga_id'))
            col_company.append(p_company.search(col.select('a')[0].get('ga_name')).group('company'))
            col_item_name.append(p_company.search(col.select('a')[0].get('ga_name')).group('item_name').lstrip())       

        for col in col_products:        
            try:
                p_price.search(col.select('p')[0].get_text()).group()
            except:
                col_sales_price.append(0)
            else:
                col_sales_price.append(int(p_price.search(col.select('p')[0].get_text()).group().replace(',','')))           

        for col in col_products:
            try:
                p_price.search(col.select('p')[1].get_text()).group()
            except IndexError:
                try:
                    p_price.search(col.select('p')[0].get_text()).group().replace(',','')
                except:
                    col_origin_price.append(0)
                else:
                    col_origin_price.append(p_price.search(col.select('p')[0].get_text()).group().replace(',',''))    
            else:
                col_origin_price.append(p_price.search(col.select('p')[1].get_text()).group().replace(',',''))

        for col in col_imgthumb:
            try:
                col.select('span')[0].get_text()
            except IndexError:
                col_discounts.append(float(0))
            else:
                col_discounts.append(float(col.select('span')[0].get_text()) * 0.01)

https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=1
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=2
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=3
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=4
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=5
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=6
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=7
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=8
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=9
https://www.baeminchan.com/sidedish/list.php?cno=46010000&page=10
https://www.baeminchan.com/maindish/list.php?cno=46030000&page=1
https://www.baeminchan.com/maindish/list.php?cno=46030000&page=2
https://www.baeminchan.com/maindish/list.php?cno=46030000&page=3
https://www.baeminchan.com/maindish/list.php?cno=46030000&page=4
https://www.baeminchan.com/maindish/list.php?cno=46030000&page=5
https://www.baeminchan.c

In [10]:
Item.objects.all().delete()

item_cnt = 1

for item_cnt in range(item_cnt, len(col_item_name)):    
    try:
        Item.objects.create(pk=item_cnt, item_name=col_item_name[item_cnt-1], company=col_company[item_cnt-1], origin_price=col_origin_price[item_cnt-1], sale_price=col_sales_price[item_cnt-1], discount_rate=col_discounts[item_cnt-1], ga_id=col_ga_id[item_cnt-1])
    except:        
        pass

굴생채무침(2~3인분) 220g 17481
고깃집 그 참기름 묵은지무침 (1~2인분) 200g 18631
고깃집 그 대파김치 (1~2인분) 200g 18626
고깃집 그 묵은지 (1~2인분) 200g 18627
꼬막무침 (2인분) 380g 17383
연근, 다시마 땅콩조림(렌곤 피너츠) (1인분) 100g 18597
돼지고기 장똑똑이 130g 18394
땅콩조림 (4인분) 160g 16710
느타리굴소스볶음 (2인분) 170g 16716
비엔나소세지야채볶음 (1~2인분) 200g 16457
표고버섯 무나물 (1인분) 150g 18596
참나물겉절이 (1인분) 100g 18606
탕평채버무리 (2인분) 250g 18557
무들깨나물 (2~3인분) 200g 18437
우엉깨범벅 (1~2인분) 150g 18365
쌀게볶음 (2인분) 150g 18425
아삭 우엉 잡채 (2인분) 400g 17071
콩조림 (1~2인분) 100g 18366
명엽채볶음 (1~2인분) 80g 17988
표고버섯 무조림(시타케니) (2인분) 150g 18598
고기엔 젓갈 3종 (2인분) 410g 18623
톳나물 된장 무침 (1인분) 90g 18220
고기엔 찍먹소스 3종 (2인분) 370g 18624
명란감자조림(멘타이코 자가모노) (2인분) 200g 18617
꼬들한 줄기상추무침 (1~2인분) 120g 18629
꽈리고추감자조림 (1~2인분) 200g 16452
버섯 가지구이 버무리 (1인분) 300g 17072
연근조림 대용량 (3~4인분) 360g 18095
아게다시두부곤약조림 (1인분) 150g 18111
달콤감자조림 (2~3인분) 200g 17376
감자소고기조림(니쿠자까) (3~4인분) 500g 18179
스팸양파볶음 (2~3인분) 250g 17744
소고기 우엉조림(규니꾸 고보 시구레) (3~4인분) 350g 18178
간장어묵볶음 (1~2인분) 200g 16455
실갈치 고추장 무침 (1인분) 90g 18222
동전가문어조림 (2~3인분) 200g 16453
매콤감자조림 

게장 야채비빔세트 (재료) (2인분) 75g 18524
매콤 황태고추장구이 (1~2인분)160g 18405
직화우엉불고기 (2인분) 300g 18564
돼지갈비찜 (1~2인분) 450g 17994
양념 깐새우장 (2인분) 250g 16299
야채고로케 (1~2인분) 4개 14618
코다리조림 (2~3인분) 270g 14762
통오징어 고추장소스구이 (2인분) 250g 18147
고보아게(우엉튀김) (1인분) 50g 18113
매콤치즈불고기(1인분) 430g 17323
통오징어 간장소스구이 (2인분) 250g 18146
김밥 속재료 8종 (5줄분량) 420g 16760
한입 어묵튀김 (2인분) 350g 18185
수제 치킨가스 (2인분) (150g*2개) 15927
시래기꽁치지짐 (2인분)  450g 17306
우삼겹 석쇠구이 냉채 (2인분) 350g 17858
고등어감자조림(2인분) 800g 16988
오리주물럭(반조리) 500g 18011
김치목살찜 (2인분) 600g 18427
미나리향 오징어초무침 (2~3인분) 300g 16998
LA갈비 550g (반조리) 12477
소고기버섯샐러드 250g 12728
매콤해물 떡볶이 (1~2인분) 550g 16135
간재미 무침 (2인분) 350g 17603
새우 야채 볶음 (2인분) 300g 18263
오징어숙회와 브로콜리 (1~2인분) 260g 16957
광양불고기 (2인분) 700g 18354
불고기 냉채 (1인분) 250g 18162
찜보리굴비 (1~2인분) 14461
제육볶음 350g 15036
매운돼지갈비찜 (2인분) 630g 18506
오븐치킨 (1~2인분) 490g 17207
우리쌀 기름떡볶이 (2인분) 750g 18398
사과소스 통삼겹살 조림 18066
홍어삼합 (3~4인분) 700g 18368
갈치고비조림 (2인분) 700g 17628
잣소스닭고기해물냉채 170g 12620
대패삼겹살 두루치기 (2인분) 350g 17857
간장 떡볶이 (2~3인분) 810g 18089
삼치 고추장소스구이 250g 

웻(WET)에이징 한우 채끝 스테이크용(1등급) (1인분) 200g 18681
웻(WET)에이징 한우 등심 스테이크용(1+등급) (1인분) 200g 18684
한우  양념불고기 (1~2인분) 300g 15187
웻(WET)에이징 한우 보섭살 스테이크용(1등급) (1인분) 200g 18683
웻(WET)에이징 한우 등심 스테이크용(1등급) (1인분) 200g 18679
SRF 와규 채끝등심 (1인분) 200g 18667
웻(WET)에이징 한우 안심 스테이크용(1등급) (1인분) 200g 18680
웻(WET)에이징 한우 채끝 스테이크용(1+등급) (1인분) 200g 18687
무항생제 한우 채끝 1등급 구이용 (1인분) 200g 18641
웻(WET)에이징 한우 안심 스테이크용(1+등급) (1인분) 200g 18685
무항생제 한우 채끝 1등급 구이용 (2~3인분) 400g 18642
SRF 와규 치마살 (1인분) 200g 18668
무항생제 한우 등심 1등급 스테이크용 300g 18115
무항생제 한우 차돌박이 1등급 300g 17818
무항생제 한우 우삼겹 1등급 300g 17817
무항생제 한우 안심 1등급 스테이크용 300g 18116
쿠로부타 흑돈 목심 (1~2인분) 300g 18673
쿠로부타 흑돈 삼겹 (1~2인분) 300g 18674
돼지 목살 반마리 오마카세 (2~3인분) 400g 18638
무항생제 한돈 벌집 삼겹살 (2~3인분) 400g 18646
무항생제 한돈 벌집 삼겹살 800g 18254
무항생제 한돈 항정살 400g 18253
무항생제 한우 안심 1등급 구이용 (1인분) 200g 18645
무항생제 한우 채끝 1등급 스테이크용 300g 18114
돼지 한마리 오마카세 (3~4인분) 700g 18659
무항생제 한돈 목살(구이용) (1인분) 200g 18644
무항생제 한돈 삼겹살(구이용) (1인분) 200g 18643
무항생제 한돈 항정살 (1인분) 200g 18647
무항생제 한돈 등갈비(구이용) 400g 17161
무항생제 한돈 목

리부트클렌즈 고급 11650
스키니클렌즈 중급 11649
애프터클렌즈 스무디 3종 세트 11646
애프터클렌즈 스무디 낫어펌킨파이 450mL 17150
애프터클렌즈 스무디 트로피컬그린 450mL 17151
트러스트 유어 골든애플 230mL 11607
트러스트 유어 골든애플 500mL 10874
트러스트 유어 오렌지 1000mL 10877
트러스트 유어 오렌지 230mL 11605
트러스트 유어 오렌지 500mL 10876
트러스트 유어 자몽 500mL 10875
트러스트 유어 캐럿 230mL 16660
트러스트 유어 탠저린 230mL 13154
그린스무디 그린팩 230mL 3병 10708
그린스무디 그린팩 350mL 3병 10707
그린스무디 컬러팩 350mL 3병 14408
그린스무디 컬러팩230mL 3병 14404
그린스무디 코코넛홍시 350mL 17164
그린스무디 크리미진셍 350mL 16088
살라미에그베이글 12470
아보카도베이글 15007
케일허머스 250mL 10711
파마산치즈허머스 250mL 10712
할라피뇨허머스 250mL 10710
한우 육포 50g 16243
미니꿀빠스 (1~2회분) 200g 17276
귀리부인 크리스피 코코넛 (1~2인분) 200g 15712
그릭요거트 무설탕 (3~4인분) 350g 18559
그릭요거트 무설탕 (5인분) 500g 18560
그릭요거트 플레인 (3~4인분) 350g 18561
그릭요거트 플레인 (5인분) 500g 18562
바닐라 그래놀라 (1~2인분) 200g 15710
호두강정 (2인분) 120g 18504
고구마맛탕 (2~3인분)210g 14434
클렌즈스무디 오키드 200mL 16248
클렌즈스무디 제이드 200mL 14047
클렌즈스무디 제이드 450mL 14054
클렌즈스무디 오리진 200mL 14046
옥수수맛탕(토모로코시 마토탄) (2인분) 200g 18618
콜드브루와 유리잔 18343
드립백 (11gX6팩) 16416
콜드브루 (16~18회분) 1L 11360
콜드브루 골드 (2~4

In [14]:
# 밑반찬
'https://www.baeminchan.com/sidedish/list.php?cno=46010000'
'https://www.baeminchan.com/sidedish/list.php?cno=46010100'
'https://www.baeminchan.com/sidedish/list.php?cno=46010200'
'https://www.baeminchan.com/sidedish/list.php?cno=46010300'
'https://www.baeminchan.com/sidedish/list.php?cno=46010400'
'https://www.baeminchan.com/sidedish/list.php?cno=46010600'
'https://www.baeminchan.com/sidedish/list.php?cno=46010700'
# 메인반찬
'https://www.baeminchan.com/maindish/list.php?cno=46030000'
'https://www.baeminchan.com/maindish/list.php?cno=46030100'
'https://www.baeminchan.com/maindish/list.php?cno=46030200'
'https://www.baeminchan.com/maindish/list.php?cno=46030300'
'https://www.baeminchan.com/maindish/list.php?cno=46030400'
'https://www.baeminchan.com/maindish/list.php?cno=46030500'
'https://www.baeminchan.com/maindish/list.php?cno=46030600'
# 국찌개탕
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''

''